# Federal_Reserve_Bank_Workspace_Final_Report
Virginia Commonwealth University

Faitha Schrader and Bryce Bowles

Spring 2020



Read data.

In [1]:
import pandas as pd

move_data = pd.read_excel("Fed_Res_w20o14.xlsx",
                       sheet_name="Moves",
                          index_col=0)
S = list(move_data)
W = list(move_data.index)
m = move_data.stack().to_dict()
A = list(m.keys())
V = W+S

distance_data = pd.read_excel("Fed_Res_w20o14.xlsx",
                       sheet_name="Distance",
                          index_col=0)
d = distance_data.stack().to_dict()
D = list(d.keys())

worker_data = pd.read_excel("Fed_Res_w20o14.xlsx",
                       sheet_name="Worker",
                          index_col=0)
rank_worker = dict(zip(move_data.index, worker_data.Rank))
team_worker = dict(zip(move_data.index, worker_data.Team))
space_data = pd.read_excel("Fed_Res_w20o14.xlsx",
                       sheet_name="Space",
                          index_col=0)
rank_space = dict(zip(move_data, space_data.Rank))

capacity = {'ACO':-1, 'A2W1':-1, 'A3W': -1, 'AEC': -7, 'AIC': -7, 'A2W2': -6, 'BCO': -1, 'B2W1':-1,'B3W':-1,'BEC':-6,'BIC':-6,'BIO1':-1,'BIO2':-1,'B2W2':-1}

b = dict(zip(W, [1]*len(W)))
b.update(capacity)

print(b)

def mybounds(model, i,j):
    return (0.0, 1.0)
def mybounds2(model, i,j,k,l):
    return (0.0, 1.0)


{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 'ACO': -1, 'A2W1': -1, 'A3W': -1, 'AEC': -7, 'AIC': -7, 'A2W2': -6, 'BCO': -1, 'B2W1': -1, 'B3W': -1, 'BEC': -6, 'BIC': -6, 'BIO1': -1, 'BIO2': -1, 'B2W2': -1}


Import Pyomo and define model.

In [2]:
from pyomo.environ import *
model = ConcreteModel()

Let $x_{ij}$ be the flow on arc $(i,j)$ for $(i,j) \in A$.

In [3]:
try:
    model.del_component(model.x)
    model.del_component(model.x_index)
    model.del_component(model.x_index_0)
    model.del_component(model.x_index_1)
except:
    print("Defining variables for the first time")
model.x = Var(A, domain=Binary, bounds=mybounds)

Defining variables for the first time


In [4]:
try: 
    model.del_component(model.y)
    model.del_component(model.y_index)
    model.del_component(model.y_index_0)
    model.del_component(model.y_index_1)
except:
    print("Defining variables for first time")
model.y = Var(A,A, domain=Binary, bounds=mybounds2)

Defining variables for first time


In [5]:
try:
    model.del_component(model.Moves_Distance)
except:
    print("Defining objective for the first time")
model.Moves_Distance = Objective(expr=sum(m[i,j]*model.x[i,j] for (i,j) in A)+sum(sum(sum(sum(d[j,l] * model.y[i,j,k,l] for i in W) for j in S) for k in W) for l in S), sense=minimize)

Defining objective for the first time


Flow Balance constraints:

$$
\sum_{j \in V: (i,j) \in A} x_{ij} - \sum_{j \in V: (j,i) \in A} x_{ji} = b_i, i \in V \mbox{ Flow Balance}
$$

In [6]:
try:
    model.del_component(model.FlowBalance)
    model.del_component(model.FlowBalance_index)
except:
    print("Defining constraints for the first time")
    
model.FlowBalance = ConstraintList()
for i in V:
    if i in W:
        model.FlowBalance.add(sum(model.x[i,j] for j in V if (i,j) in A) - sum(model.x[j,i] for j in V if (j,i) in A) == b[i])
    else:
        model.FlowBalance.add(sum(model.x[i,j] for j in V if (i,j) in A) - sum(model.x[j,i] for j in V if (j,i) in A) >= b[i])


Defining constraints for the first time


In [7]:
try:
    model.del_component(model.Flip_on)
    model.del_component(model.Flip_on_index)
except:
    print("Defining constraints for the first time")
    
model.Flip_on = ConstraintList()
for i in W:
    for j in S:
        for k in W:
            for l in S:
                if team_worker[i] == team_worker[k]: model.Flip_on.add(model.x[i,j] + model.x [k,l] <= model.y[i,j,k,l] + 1)
                else: model.Flip_on.add(model.y[i,j,k,l] == 0)

Defining constraints for the first time


In [8]:
try:
    model.del_component(model.RankConstraint)
    model.del_component(model.RankConstraint_index)
except:
    print("Defining constraints for the first time")
    
model.RankConstraint = ConstraintList()
for i in W:
    for j in S:
        if rank_worker[i] != rank_space [j]: model.RankConstraint.add(model.x[i,j] ==0)

Defining constraints for the first time


Specify solver and solve.

In [9]:
solver = SolverFactory("glpk")
status = solver.solve(model)

In [10]:
print("Status = %s" % status)

Status = 
Problem: 
- Name: unknown
  Lower bound: 1240.0
  Upper bound: 1240.0
  Number of objectives: 1
  Number of constraints: 78663
  Number of variables: 78681
  Number of nonzeros: 138493
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 131
      Number of created subproblems: 131
  Error rc: 0
  Time: 0.8706469535827637
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [11]:
for (i,j) in A:
    print("%s = %f" % (model.x[i,j], value(model.x[i,j])))

print("Moves_Distance = %f" % value(model.Moves_Distance))
total_moves = int(sum(m[i,j]* value(model.x[i,j] )for (i,j) in A))
total_distance = int(value(model.Moves_Distance) - total_moves)
print("total_moves = %i" % total_moves)
print("total_distance = %i ft." % total_distance)
team_distance = int((sum(d[j,l] * value(model.y[i,j,k,l] for i in W) for j in S) for k in W) for l in S)
print("Team %i total distance: %i ft." % team_distance)



x[1,ACO] = 0.000000
x[1,A2W1] = 0.000000
x[1,A3W] = 0.000000
x[1,AEC] = 0.000000
x[1,AIC] = 0.000000
x[1,A2W2] = 0.000000
x[1,BCO] = 0.000000
x[1,B2W1] = 0.000000
x[1,B3W] = 1.000000
x[1,BEC] = 0.000000
x[1,BIC] = 0.000000
x[1,BIO1] = 0.000000
x[1,BIO2] = 0.000000
x[1,B2W2] = 0.000000
x[2,ACO] = 0.000000
x[2,A2W1] = 0.000000
x[2,A3W] = 0.000000
x[2,AEC] = 0.000000
x[2,AIC] = 0.000000
x[2,A2W2] = 0.000000
x[2,BCO] = 0.000000
x[2,B2W1] = 0.000000
x[2,B3W] = 0.000000
x[2,BEC] = 1.000000
x[2,BIC] = 0.000000
x[2,BIO1] = 0.000000
x[2,BIO2] = 0.000000
x[2,B2W2] = 0.000000
x[3,ACO] = 0.000000
x[3,A2W1] = 0.000000
x[3,A3W] = 0.000000
x[3,AEC] = 0.000000
x[3,AIC] = 0.000000
x[3,A2W2] = 0.000000
x[3,BCO] = 0.000000
x[3,B2W1] = 0.000000
x[3,B3W] = 0.000000
x[3,BEC] = 1.000000
x[3,BIC] = 0.000000
x[3,BIO1] = 0.000000
x[3,BIO2] = 0.000000
x[3,B2W2] = 0.000000
x[4,ACO] = 0.000000
x[4,A2W1] = 0.000000
x[4,A3W] = 0.000000
x[4,AEC] = 0.000000
x[4,AIC] = 0.000000
x[4,A2W2] = 0.000000
x[4,BCO] = 0.000000


TypeError: int() argument must be a string, a bytes-like object or a number, not 'generator'